### Exercise 1: aka AnalyzeGraph

In [75]:
import time
import pygraphblas as grb
from pygraphblas.demo.gviz import draw, draw_op

In [76]:
def BFS(graph, src_node):
    num_nodes = graph.nrows
    w = grb.Vector.sparse(grb.types.BOOL, num_nodes) #wavefront
    v = grb.Vector.sparse(grb.types.BOOL, num_nodes) #visited
    w[src_node] = True
    
    with grb.BOOL.LOR_LAND:
        while w.nvals > 0:
            v.assign_scalar(True, mask=w)
            w.vxm(graph, mask=v, out=w, desc=grb.RC)
            
    return v

def connected_components(graph):
    num_nodes = graph.nrows
    cc_ids    = grb.Vector.sparse(grb.types.UINT64, num_nodes)
    num_ccs   = 0
    
    for src_node in range(num_nodes):
        if cc_ids.get(src_node) == None:
            #print("Processing node", src_node)
            visited = BFS(graph, src_node) # traverse from src_node marking all reachable nodes
            
            #cc_ids[visited] = src_node
            cc_ids.assign_scalar(src_node, mask=visited)
            num_ccs += 1
            
    return num_ccs, cc_ids

In [77]:
def pagerank(A, damping = 0.85, itermax = 100):
    d = A.reduce_vector(grb.types.FP32.PLUS_MONOID)
    
    n = A.nrows
    r = grb.Vector.sparse(grb.types.FP32, n)
    t = grb.Vector.sparse(grb.types.FP32, n)
    d.assign_scalar(damping, accum=grb.types.FP32.DIV)
    r[:] = 1.0 / n
    teleport = (1 - damping) / n
    tol = 1e-4
    rdiff = 1.0
    for i in range(itermax):
        # swap t and r
        temp = t ; t = r ; r = temp
        w = t / d
        r[:] = teleport
        A.mxv(w,
              out=r,
              accum=grb.types.FP32.PLUS,
              semiring=grb.types.FP32.PLUS_SECOND,
              desc=grb.T0)
        t -= r
        t.apply(grb.types.FP32.ABS, out=t)
        rdiff = t.reduce_float()
        if rdiff <= tol:
            break
    return r

In [84]:
def AnalyzeGraph(pathname):
    #================================================================
    print("*** Step 1: loading input graph:", pathname)
    with open(pathname, 'r') as f:
        t0 = time.time()
        M = grb.Matrix.from_mm(f, grb.types.UINT64)
        t1 = time.time()
        print("*** Step 1: Elapsed time: %s sec." % (t1 - t0))
    
    #================================================================
    print("*** Step 2: compute some basic statistics")
    t0 = time.time()
    num_rows = M.nrows
    num_cols = M.ncols
    num_vals = M.nvals
    
    degree = M.reduce_vector(grb.types.UINT64.PLUS_MONOID)
    
    max_degree = degree.reduce_int(grb.types.UINT64.MAX_MONOID)
    min_degree = degree.reduce_int(grb.types.UINT64.MIN_MONOID)
    t1 = time.time()
    
    print("*** Step 2: elapsed time: %s sec." % (t1 - t0))
    
    print("Num nodes: ", num_rows)
    print("Num edges: ", num_vals)
    print("Avg degree:", float(num_vals)/float(num_rows))
    print("Max degree:", max_degree)
    print("Min degree:", min_degree)
    
    target_ID = 0
    for ix in range(num_rows):
        if degree.get(ix) == max_degree:
            target_ID = ix
            print("Node with max degree:", ix)
            
    #================================================================
    print("*** Step 3: Run a connected components algorithm.")
    t0 = time.time()
    num_ccs, components = connected_components(M)
    t1 = time.time()
    print("*** Step 3: elapsed time: %s sec." % (t1 - t0))

    print("Found", num_ccs, "connected components")
    component_ID = components.get(target_ID)
    print("Node", target_ID, "component ID is", component_ID)
    
    #================================================================
    print("*** Step 4: Find all the nodes from the target ID's cluster.")
    t0 = time.time()
    cluster_mask = components.select('==', component_ID)
    #print(cluster_mask)
    
    # Get the number of elements in the mask and extract the indices
    # cluster_indices = list(cluster_mask.indexes)
    [cluster_indices, cluster_vals] = cluster_mask.to_lists()
    component_size = len(cluster_indices)
    t1 = time.time()
    print("*** Step 4: elapsed time: %s sec." % (t1 - t0))
    print("Number of nodes in target ID's component:", len(cluster_indices))
    #print("Component members:", cluster_indices)

    #================================================================
    print("*** Step 5: extract and perform PageRank on the target component.")
    t0 = time.time()
    A_comp = M.extract_matrix(cluster_indices, cluster_indices)
    #print(A_comp)
    pr = pagerank(A_comp)
    t1 = time.time()
    print("*** Step 5: elapsed time: %s sec." % (t1 - t0))
    #print(pr)
    max_rank = pr.reduce_float(grb.types.FP32.MAX_MONOID)
    min_rank = pr.reduce_float(grb.types.FP32.MIN_MONOID)
    print("min, max ranks:", min_rank, max_rank)
    
    [pr_indices, pr_ranks] = pr.to_lists()
    for ix in range(len(pr_indices)):
        if (pr_ranks[ix] == max_rank):
            print("Author with highest rank:", cluster_indices[pr_indices[ix]], pr_indices[ix])
        if (pr_ranks[ix] == min_rank):
            print("Author with lowest rank: ", cluster_indices[pr_indices[ix]], pr_indices[ix])
        if (cluster_indices[pr_indices[ix]] == target_ID):
            print("Author", target_ID, "rank: ", pr_ranks[ix], pr_indices[ix])
    return A_comp, pr_indices, pr_ranks

In [85]:
sub_graph, indices, ranks = AnalyzeGraph('./hpec_coauthors.mtx')

*** Step 1: loading input graph: ./hpec_coauthors.mtx
*** Step 1: Elapsed time: 0.03614330291748047 sec.
*** Step 2: compute some basic statistics
*** Step 2: elapsed time: 0.0005049705505371094 sec.
Num nodes:  1747
Num edges:  10072
Avg degree: 5.76531196336577
Max degree: 461
Min degree: 1
Node with max degree: 800
*** Step 3: Run a connected components algorithm.
*** Step 3: elapsed time: 0.026781082153320312 sec.
Found 246 connected components
Node 800 component ID is 0
*** Step 4: Find all the nodes from the target ID's cluster.
*** Step 4: elapsed time: 0.0007436275482177734 sec.
Number of nodes in target ID's component: 822
*** Step 5: extract and perform PageRank on the target component.
*** Step 5: elapsed time: 0.006418943405151367 sec.
min, max ranks: 0.00019327869813423604 0.009313824586570263
Author 800 rank:  0.0075648571364581585 357
Author with lowest rank:  1094 513
Author with highest rank: 1424 676
